In [2]:
import numpy as np
from collections import defaultdict

# ===============================
# ÉTAPE 1 : Chargement du dataset traité
# ===============================
data = np.load("/kaggle/input/daicwoz/processed_dataset.npz", allow_pickle=True)
X_audio = data["X_audio"]
X_text = data["X_text"]
y = data["y"]
ids = data["ids"]

# ===============================
# ÉTAPE 2 : Séparation des indices selon les classes (0 et 1)
# ===============================
class_indices = defaultdict(list)
for i, label in enumerate(y):
    class_indices[int(label)].append(i)

# ===============================
# ÉTAPE 3 : Calcul du nombre d'exemples pour dev et test
# On utilise 10% pour dev et 10% pour test, de manière équilibrée
# ===============================
min_class = min(len(class_indices[0]), len(class_indices[1]))
n_dev = n_test = min_class // 5  # 10% par classe

# ===============================
# ÉTAPE 4 : Construction des indices pour chaque split
# ===============================
dev_idx = class_indices[0][:n_dev] + class_indices[1][:n_dev]
test_idx = class_indices[0][n_dev:n_dev + n_test] + class_indices[1][n_dev:n_dev + n_test]
train_idx = list(set(range(len(y))) - set(dev_idx) - set(test_idx))

# ===============================
# ÉTAPE 5 : Fonction utilitaire pour extraire les données selon les indices
# ===============================
def extract(indices):
    return (
        np.array([X_audio[i] for i in indices], dtype=np.float32),
        np.array([X_text[i] for i in indices], dtype=np.float32),
        np.array([y[i] for i in indices], dtype=np.float32),
        np.array([ids[i] for i in indices])
    )

# ===============================
# ÉTAPE 6 : Extraction des ensembles train/dev/test
# ===============================
X_audio_train, X_text_train, y_train, ids_train = extract(train_idx)
X_audio_dev, X_text_dev, y_dev, ids_dev = extract(dev_idx)
X_audio_test, X_text_test, y_test, ids_test = extract(test_idx)

In [3]:
# ===============================
# ÉTAPE 7 : Sauvegarde du dataset final équilibré
# ===============================
np.savez_compressed(
    '/kaggle/working/dataset_balanced.npz',
    X_audio_train=X_audio_train,
    X_text_train=X_text_train,
    y_train=y_train,
    ids_train=ids_train,
    X_audio_dev=X_audio_dev,
    X_text_dev=X_text_dev,
    y_dev=y_dev,
    ids_dev=ids_dev,
    X_audio_test=X_audio_test,
    X_text_test=X_text_test,
    y_test=y_test,
    ids_test=ids_test
)

# ===============================
# ÉTAPE 8 : Sauvegarde séparée des IDs du test set
# ===============================
np.save("/kaggle/working/test_ids.npy", ids_test)

# ===============================
# ÉTAPE 9 : Confirmation
# ===============================
print("✅ Dataset équilibré sauvegardé")
print("📁 test_ids.npy sauvegardé avec succès")

✅ Dataset équilibré sauvegardé
📁 test_ids.npy sauvegardé avec succès


In [5]:
import numpy as np
from collections import Counter

# Charger les données
data = np.load("/kaggle/working/dataset_balanced.npz", allow_pickle=True)

# Extraire les labels
y_train = data["y_train"]
y_dev = data["y_dev"]
y_test = data["y_test"]

# Affichage du nombre d'exemples
print("📊 Nombre d'exemples :")
print(f"Train : {len(y_train)}")
print(f"Dev   : {len(y_dev)}")
print(f"Test  : {len(y_test)}")

# Vérifier l’équilibrage des classes
print("\n⚖️  Répartition des classes :")
print("Train :", dict(Counter(y_train)))
print("Dev   :", dict(Counter(y_dev)))
print("Test  :", dict(Counter(y_test)))

📊 Nombre d'exemples :
Train : 8700
Dev   : 1392
Test  : 1392

⚖️  Répartition des classes :
Train : {0.0: 6609, 1.0: 2091}
Dev   : {0.0: 696, 1.0: 696}
Test  : {0.0: 696, 1.0: 696}
